## Установка и первый тест

```python
# Установка: pip install pytest
# Запуск: pytest test_example.py

def add(a, b):
    return a + b

def test_add():
    assert add(2, 3) == 5
    assert add(-1, 1) == 0
    assert add(0, 0) == 0
```

## Базовые assertions

```python
def test_assertions():
    # Основные проверки
    assert 2 + 2 == 4
    assert 5 > 3
    assert "hello" in "hello world"
    assert [1, 2, 3] != [1, 2]
    
    # С кастомным сообщением
    assert len("test") == 4, "Строка должна содержать 4 символа"
    
    # Проверка типов
    assert isinstance(42, int)
```

## Тестирование исключений с pytest.raises

```python
# Контекст-менеджер

import pytest

def divide(a, b):
    if b == 0:
        raise ValueError("Деление на ноль")
    return a / b

def test_divide_exceptions():
    # Проверка типа исключения
    with pytest.raises(ValueError):
        divide(10, 0)
    
    # Проверка сообщения исключения
    with pytest.raises(ValueError, match=r".*ноль.*"):
        divide(5, 0)
    
    # Доступ к объекту исключения
    with pytest.raises(ValueError) as exc_info:
        divide(1, 0)
    assert "ноль" in str(exc_info.value)
```



## Базовые fixtures

```python
# Fixtures — функции для подготовки тестовых данных. Внедрение зависимостей через аргументы функций.

import pytest

@pytest.fixture
def sample_data():
    return {"name": "Alice", "age": 30, "email": "alice@test.com"}

@pytest.fixture
def temp_list():
    return [1, 2, 3, 4, 5]

def test_user_data(sample_data):
    assert sample_data["name"] == "Alice"
    assert sample_data["age"] == 30
    assert "@" in sample_data["email"]

def test_list_operations(temp_list):
    assert len(temp_list) == 5
    assert sum(temp_list) == 15
```



## Fixture scopes (области видимости)

```python
# Scope определяет время жизни fixture: function (по умолчанию), class, module, package, session.

import pytest

@pytest.fixture(scope="session")
def database_connection():
    """Создается один раз за всю сессию тестов"""
    print("Подключение к БД")
    connection = {"status": "connected"}
    yield connection
    print("Отключение от БД")

@pytest.fixture(scope="function")  # По умолчанию
def fresh_data():
    """Создается для каждого теста"""
    return {"counter": 0}

def test_db_operations(database_connection, fresh_data):
    assert database_connection["status"] == "connected"
    fresh_data["counter"] += 1
    assert fresh_data["counter"] == 1
```



## Setup/Teardown с yield

```python
# yield в fixtures обеспечивает setup (до yield) и teardown (после yield) для очистки ресурсов.

import pytest
import tempfile
import os

@pytest.fixture
def temp_file():
    # Setup: создание временного файла
    fd, filepath = tempfile.mkstemp(suffix='.txt')
    with os.fdopen(fd, 'w') as f:
        f.write("test content")
    
    yield filepath  # Возвращаем путь к файлу тесту
    
    # Teardown: удаление файла после теста
    if os.path.exists(filepath):
        os.unlink(filepath)

def test_file_operations(temp_file):
    assert os.path.exists(temp_file)
    with open(temp_file, 'r') as f:
        content = f.read()
    assert content == "test content"
```



## conftest.py — общие fixtures

```python
# conftest.py делает fixtures доступными во всех тестах директории и поддиректорий без импорта.

# conftest.py
import pytest

@pytest.fixture(scope="session")
def app_config():
    return {
        "api_url": "https://api.test.com",
        "timeout": 30,
        "debug": True
    }

@pytest.fixture
def authenticated_user():
    return {"id": 123, "role": "admin", "token": "abc123"}

# test_api.py (fixtures автоматически доступны)
def test_api_call(app_config, authenticated_user):
    assert app_config["api_url"].startswith("https://")
    assert authenticated_user["role"] == "admin"
```



## Parametrize — множественные тестовые случаи

```python
# @pytest.mark.parametrize запускает один тест с разными наборами параметров. Сокращает дублирование кода.

import pytest

@pytest.mark.parametrize("a,b,expected", [
    (2, 3, 5),
    (-1, 1, 0),
    (0, 0, 0),
    (10, -5, 5)
])
def test_addition(a, b, expected):
    assert a + b == expected

@pytest.mark.parametrize("text,length", [
    ("hello", 5),
    ("", 0),
    ("Python", 6)
])
def test_string_length(text, length):
    assert len(text) == length
```



## Parametrize с ids для читаемости

```python
# ids делают названия параметризованных тестов более понятными в выводе.

import pytest

@pytest.mark.parametrize("value,expected", [
    (0, False),
    (1, True),
    (-1, True),
    ([], False),
    ([1, 2], True)
], ids=["zero", "positive", "negative", "empty_list", "non_empty_list"])
def test_truthiness(value, expected):
    assert bool(value) == expected

# Использование функции для генерации ids
def idfn(val):
    if isinstance(val, list):
        return f"list_len_{len(val)}"
    return str(val)

@pytest.mark.parametrize("data", [[], [1], [1,2,3]], ids=idfn)
def test_list_processing(data):
    assert isinstance(data, list)
```



## Fixture parametrization

```python
# Параметризация fixtures позволяет тестировать с разными конфигурациями окружения.

import pytest

@pytest.fixture(params=["sqlite", "postgresql", "mysql"])
def database_type(request):
    return request.param

@pytest.fixture(params=[True, False])
def cache_enabled(request):
    return request.param

def test_database_connection(database_type, cache_enabled):
    # Тест выполнится для всех комбинаций параметров
    config = {
        "db_type": database_type,
        "cache": cache_enabled
    }
    
    assert config["db_type"] in ["sqlite", "postgresql", "mysql"]
    assert isinstance(config["cache"], bool)
```



## Marks — skip и xfail

```python
# Skip пропускает тесты при определенных условиях. Xfail помечает ожидаемые неудачи.

import pytest
import sys

@pytest.mark.skip(reason="Функция еще не реализована")
def test_future_feature():
    assert False

@pytest.mark.skipif(sys.platform == "win32", reason="Не работает на Windows")
def test_unix_specific():
    assert True

@pytest.mark.xfail(reason="Известный баг #123")
def test_known_issue():
    assert 1 == 2  # Ожидается неудача

@pytest.mark.xfail(sys.version_info < (3, 12), reason="Требует Python 3.12+")
def test_new_python_feature():
    # Тест новой функции Python 3.12+
    pass
```



## Пользовательские marks

```python
# Создание собственных marks для группировки и фильтрации тестов.

import pytest

# В pytest.ini или pyproject.toml:
# [tool.pytest.ini_options]
# markers = [
#     "slow: marks tests as slow",
#     "integration: marks tests as integration tests",
#     "unit: marks tests as unit tests"
# ]

@pytest.mark.slow
def test_heavy_computation():
    # Медленный тест
    result = sum(range(1000000))
    assert result > 0

@pytest.mark.integration
def test_database_integration():
    # Интеграционный тест
    assert True

@pytest.mark.unit
def test_simple_function():
    # Быстрый unit-тест
    assert 2 + 2 == 4

# Запуск: pytest -m "not slow" (исключить медленные тесты)
# pytest -m "integration" (только интеграционные)
```



## Monkeypatch для мокинга

```python
# Monkeypatch временно заменяет атрибуты, функции или переменные окружения для изоляции тестов.

import pytest
import os

def get_config():
    return {
        "debug": os.getenv("DEBUG", "false").lower() == "true",
        "host": os.getenv("HOST", "localhost")
    }

def external_api_call():
    return {"status": "real_call"}

def test_config_with_env(monkeypatch):
    # Подмена переменных окружения
    monkeypatch.setenv("DEBUG", "true")
    monkeypatch.setenv("HOST", "testhost")
    
    config = get_config()
    assert config["debug"] is True
    assert config["host"] == "testhost"

def test_api_mock(monkeypatch):
    # Подмена функции
    def mock_api():
        return {"status": "mocked"}
    
    monkeypatch.setattr("__main__.external_api_call", mock_api)
    result = external_api_call()
    assert result["status"] == "mocked"
```



## Автоматические fixtures (autouse)

```python
# autouse=True заставляет fixture выполняться автоматически для всех тестов в области видимости.

import pytest
import logging

@pytest.fixture(autouse=True, scope="function")
def setup_logging():
    """Настраивает логирование для каждого теста"""
    logging.basicConfig(level=logging.DEBUG)
    yield
    # Очистка после теста
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

@pytest.fixture(autouse=True, scope="session")
def test_environment():
    """Настройка тестового окружения один раз за сессию"""
    print("\n=== Начало тестовой сессии ===")
    yield
    print("\n=== Конец тестовой сессии ===")

def test_with_auto_logging():
    logging.info("Логирование настроено автоматически")
    assert True
```



## Зависимые fixtures

```python
# Fixtures могут зависеть друг от друга, создавая цепочки инициализации.

import pytest

@pytest.fixture(scope="session")
def database():
    """Базовое подключение к БД"""
    db = {"connection": "active", "tables": []}
    yield db
    db["connection"] = "closed"

@pytest.fixture
def user_table(database):
    """Создает таблицу пользователей"""
    table_name = "users"
    database["tables"].append(table_name)
    yield {"name": table_name, "rows": []}
    # Очистка таблицы
    database["tables"].remove(table_name)

@pytest.fixture
def sample_user(user_table):
    """Создает тестового пользователя"""
    user = {"id": 1, "name": "Test User"}
    user_table["rows"].append(user)
    yield user
    # Удаление пользователя
    user_table["rows"].clear()

def test_user_operations(sample_user, user_table, database):
    assert database["connection"] == "active"
    assert user_table["name"] == "users"
    assert sample_user["name"] == "Test User"
    assert len(user_table["rows"]) == 1
```



## Временные файлы и директории

```python
# tmp_path и tmp_path_factory — встроенные fixtures для работы с временными файлами.

import pytest

def test_temp_file(tmp_path):
    """tmp_path предоставляет уникальную временную директорию для теста"""
    # Создание временного файла
    test_file = tmp_path / "test.txt"
    test_file.write_text("Hello, World!")
    
    assert test_file.read_text() == "Hello, World!"
    assert test_file.exists()
    # Файл автоматически удаляется после теста

def test_temp_directory_structure(tmp_path):
    """Создание структуры директорий"""
    subdir = tmp_path / "subdir"
    subdir.mkdir()
    
    (subdir / "file1.txt").write_text("Content 1")
    (subdir / "file2.txt").write_text("Content 2")
    
    files = list(subdir.glob("*.txt"))
    assert len(files) == 2

@pytest.fixture(scope="session")
def shared_temp_dir(tmp_path_factory):
    """Общая временная директория для всей сессии"""
    return tmp_path_factory.mktemp("shared_data")
```



## Захват stdout/stderr (capsys)

```python
# capsys позволяет захватывать и тестировать вывод в stdout/stderr.

import sys

def print_greeting(name):
    print(f"Hello, {name}!")
    
def print_error(message):
    print(f"ERROR: {message}", file=sys.stderr)

def test_stdout_capture(capsys):
    """Тестирование вывода в stdout"""
    print_greeting("Alice")
    
    captured = capsys.readouterr()
    assert captured.out == "Hello, Alice!\n"
    assert captured.err == ""

def test_stderr_capture(capsys):
    """Тестирование вывода в stderr"""
    print_error("Something went wrong")
    
    captured = capsys.readouterr()
    assert "ERROR: Something went wrong" in captured.err
    assert captured.out == ""

def test_mixed_output(capsys):
    """Тестирование смешанного вывода"""
    print("Normal output")
    print_error("Error output")
    
    captured = capsys.readouterr()
    assert "Normal output" in captured.out
    assert "Error output" in captured.err
```



## Класс-базированные тесты

```python
# Группировка связанных тестов в классы для лучшей организации. Классы должны начинаться с Test.

import pytest

class TestCalculator:
    """Группа тестов для калькулятора"""
    
    @pytest.fixture(autouse=True)
    def setup_method(self):
        """Выполняется перед каждым методом теста"""
        self.calculator = {"memory": 0}
    
    def test_addition(self):
        result = 2 + 3
        assert result == 5
        self.calculator["memory"] = result
    
    def test_subtraction(self):
        result = 10 - 4
        assert result == 6
    
    @pytest.mark.parametrize("a,b,expected", [
        (6, 2, 3),
        (15, 3, 5),
        (20, 4, 5)
    ])
    def test_division(self, a, b, expected):
        assert a / b == expected

class TestStringOperations:
    """Отдельная группа для строковых операций"""
    
    def test_uppercase(self):
        assert "hello".upper() == "HELLO"
    
    def test_lowercase(self):
        assert "WORLD".lower() == "world"
```



## Конфигурация pytest.ini/pyproject.toml

```python
# Конфигурация pytest через файлы pytest.ini или pyproject.toml для настройки поведения по умолчанию.

# pytest.ini
[tool.pytest.ini_options]
minversion = "6.0"
addopts = [
    "--strict-markers",
    "--strict-config",
    "-ra"  # показать краткую сводку всех результатов
]
testpaths = ["tests"]
python_files = ["test_*.py", "*_test.py"]
python_classes = ["Test*"]
python_functions = ["test_*"]
markers = [
    "slow: marks tests as slow (deselect with '-m \"not slow\"')",
    "integration: marks tests as integration tests",
    "unit: marks tests as unit tests",
    "smoke: marks tests as smoke tests"
]

# Пример использования в тестах
@pytest.mark.smoke
def test_basic_functionality():
    """Быстрый smoke-тест основной функциональности"""
    assert True

# Запуск с фильтрацией: pytest -m smoke
```



## Плагины и расширения

```python
# Pytest имеет богатую экосистему плагинов. Установка через pip, автоматическое подключение.

# Полезные плагины:
# pip install pytest-cov pytest-xdist pytest-mock pytest-html

# pytest-cov: покрытие кода
# pytest test_module.py --cov=my_module --cov-report=html

# pytest-xdist: параллельный запуск тестов
# pytest -n 4  # запуск на 4 процессорах

# pytest-mock: улучшенный мокинг
def test_with_mock(mocker):
    mock_func = mocker.patch('os.path.exists')
    mock_func.return_value = True
    
    import os.path
    assert os.path.exists('/fake/path') is True
    mock_func.assert_called_once_with('/fake/path')

# pytest-html: HTML-отчеты
# pytest --html=report.html --self-contained-html

def test_for_html_report():
    """Тест будет включен в HTML-отчет"""
    assert "pytest" in "pytest is awesome"
```